# Prepare Workspace

## Install required modules

In [1]:
#!pip install geopandas

## Load required modules

In [2]:
import pandas as pd
import numpy as np

import datetime

import matplotlib.pyplot as plt
import seaborn as sns

#from sklearn.preprocessing import MinMaxScaler

# https://scikit-learn.org/stable/modules/impute.html
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
import geopandas as gpd

## Load files

### ACS, SVI, Medical Facilities

In [4]:
acs = pd.read_csv('ACS_Tracts.csv', low_memory=False)
svi = pd.read_csv('SVI2020_US_Tract.csv', low_memory=False)

In [5]:
facilities = pd.read_csv("medical_facilities_tracts.csv")#gpd.read_file("./Tracts_Medical_Facilities/Tracts_Medical_Facilities.shp")

In [6]:
#facilities["FIPS"]=facilities["STATEFP"]+ facilities["COUNTYFP"] + facilities["Geo_FIPS"]
#facilities["FIPS"] = facilities.FIPS.astype(int)

In [7]:
facilities.head()

,Join_ID,FIPS,Point Coun
0,1,1001020100,NaN
1,2,1001020200,NaN
2,3,1001020300,5.0
3,4,1001020400,6.0
4,5,1001020501,4.0


In [8]:
#facilities.to_excel("facilities.xlsx")
facilities = facilities[["FIPS", "Point Coun"]].copy()

In [9]:
transportation = gpd.read_file("../Tracts_Transportation/Tracts_Transportation.shp", include_fields=["FIPS", "Join_ID", "Point_Coun"])

In [10]:
#transportation["FIPS"]=transportation["STATEFP"]+ transportation["COUNTYFP"] + transportation["Geo_FIPS"]
#transportation["FIPS"] = transportation.FIPS.astype(int)
transportation = transportation[["FIPS", "Point_Coun"]].copy()

In [11]:
#transportation["Point_Coun"].sum()
transportation[transportation["Point_Coun"]>0]

,FIPS,Point_Coun
471,01073000100,16
472,01073000300,26
473,01073000400,23
474,01073000500,29
475,01073000700,29
...,...,...
83917,55139001801,23
83918,55139001803,16
83920,55139001900,3
83938,55139003300,1


# Prepare Data

# Join with ACS and SVI

## join with ACS

In [12]:
acs.drop(0, inplace=True)
acs["FIPS"] = acs["FIPS"].astype(int)

In [13]:
acs["State Postal Abbreviation"].unique()

array(['al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga',
       'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma',
       'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny',
       'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
       'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy', 'pr'], dtype=object)

In [14]:
mask = ((acs["State Postal Abbreviation"]!="ak") & (acs["State Postal Abbreviation"]!="hi"))
#ak=Alaska, hi=Hawaii

In [15]:
mask

1        True
2        True
3        True
4        True
5        True
         ... 
85391    True
85392    True
85393    True
85394    True
85395    True
Name: State Postal Abbreviation, Length: 85395, dtype: bool

In [16]:
acs = acs[mask]

In [17]:
acs["State Postal Abbreviation"].unique()

array(['al', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga', 'id',
       'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi', 'mn',
       'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny', 'nc', 'nd',
       'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt',
       'va', 'wa', 'wv', 'wi', 'wy', 'pr'], dtype=object)

In [18]:
#df=acs.set_index('FIPS').join(svi.set_index('FIPS'))#.set_index('FIPS'))
#df = df.sort_values(by="FIPS", ascending=False)
#df=df.reset_index()

In [19]:
#np.issubdtype(df["FIPS"].dtype, np.number)
#df[["FIPS"]].applymap(np.isreal)
#df[["FIPS"]].applymap(lambda x: isinstance(int(x), (int, float)))

## join with SVI

In [20]:
svi.replace([-999], np.nan, inplace=True) # -999 encodes NAN

In [21]:
#df_merged_copy = df_merged.copy()
#df_merged = df.copy()
#df=df.reset_index()

In [22]:
numeric_cols = list(svi.select_dtypes(include="number"))
svi_imputed = svi.copy()

In [23]:
#imp = IterativeImputer(missing_values=-999, max_iter=10, random_state=0)
#imp.fit(svi[numeric_cols])
#IterativeImputer(random_state=0)
#svi_imputed[numeric_cols] = imp.transform(svi[numeric_cols])

In [24]:
df=acs.set_index('FIPS').join(svi.set_index('FIPS'))#.set_index('FIPS'))
df = df.sort_values(by="FIPS", ascending=False)
df=df.reset_index()
#df['FIPS']=df['FIPS'].astype('int64')
#df_merged=df.set_index('FIPS').join(svi_imputed.set_index('FIPS'))

In [25]:
#df_merged

In [26]:
#df_merged.drop(["level_0", "index"], inplace=True)

### join with medical facilities

In [27]:
#df_merged['County of current residence']=df_merged['County of current residence'].astype('int64').copy()
facilities['FIPS']=facilities['FIPS'].astype('int64')#.copy()
facilities.set_index("FIPS", inplace = True)
df.set_index('FIPS', inplace=True)

In [28]:
facilities = facilities.fillna(0)

In [29]:
facilities.index
#facilities.reset_index(inplace=True)

Int64Index([ 1001020100,  1001020200,  1001020300,  1001020400,  1001020501,
             1001020502,  1001020503,  1001020600,  1001020700,  1001020801,
            ...
            56041975202, 56041975301, 56041975302, 56041975401, 56041975402,
            56043000200, 56043000301, 56043000302, 56045951100, 56045951300],
           dtype='int64', name='FIPS', length=84122)

In [30]:
#facilities[facilities["FIPS"]==15007040500]

In [31]:
facilities.loc[15007040500]

Point Coun    8.0
Name: 15007040500, dtype: float64

In [32]:
#df.loc[15007040500]

In [33]:
df_merged = df.copy()

In [34]:
df_merged=df_merged.join(facilities)#.set_index('FIPS'))

In [35]:
rename = {'Point Coun': "Count_Medical_Facilities"}
df_merged.rename(columns=rename, inplace=True)
df_merged.reset_index(inplace=True)

In [36]:
df_merged_copy = df_merged.copy()
df_merged = df_merged_copy.copy()

In [37]:
df_merged["Count_Medical_Facilities"].sum()

177767.0

In [38]:
df_merged["Count_Medical_Facilities"].replace([np.nan], 0, inplace=True)

In [39]:
#list(acs)
#acs[(acs['County of current residence'] == "007") & (acs['State (FIPS Code)'] == "15") & (acs['Census Tract']=="040702")].T


### join transportation

In [40]:
transportation['FIPS']=transportation['FIPS'].astype('int64').copy()
transportation.set_index("FIPS", inplace=True)
df_merged.set_index("FIPS", inplace=True)

In [41]:
#transportation.index

In [42]:
#transportation.loc[15003009603]

In [43]:
df_merged = df_merged.join(transportation)
rename    = {'Point_Coun': "Count_Transportation_Stops"}
df_merged.rename(columns=rename, inplace=True)
#df_merged.reset_index(inplace=True)

In [44]:
df_merged["Count_Transportation_Stops"].isna().sum()

1248

In [45]:
df_merged["Count_Transportation_Stops"].replace([np.nan], 0, inplace=True)

In [46]:
#df_merged.loc[15007040500]

## Calculate columns

In [47]:
df_merged[['Total Population: Under 5 Years',
           'Total Population: 5 to 9 Years',
           'Total Population: 10 to 14 Years',
           'Total Population: 15 to 17 Years',
           'Total Population:',
          'Population 25 Years and Over: Some College or More',
          'Population 25 Years and Over: Doctorate Degree',
          'Population 25 Years and Over:',
          "Population 25 Years and Over: High School Graduate or More (Includes Equivalency)",'Population 25 Years and Over: Less than High School', 'Total Population: 18 to 24 Years','Total Population: 25 to 34 Years','Total Population: 35 to 44 Years','Total Population: 45 to 54 Years','Total Population: 55 to 64 Years','Total Population: 65 to 74 Years','Total Population: 75 to 84 Years','Total Population: 85 Years and Over']]=df_merged[['Total Population: Under 5 Years',
                                                                         'Total Population: 5 to 9 Years',
                                                                         'Total Population: 10 to 14 Years',
                                                                         'Total Population: 15 to 17 Years',
                                                                         'Total Population:',
                                                                                                           'Population 25 Years and Over: Some College or More',
                                                                                                           'Population 25 Years and Over: Doctorate Degree',
                                                                                                           'Population 25 Years and Over:',
                                                                                                          "Population 25 Years and Over: High School Graduate or More (Includes Equivalency)",'Population 25 Years and Over: Less than High School','Total Population: 18 to 24 Years','Total Population: 25 to 34 Years','Total Population: 35 to 44 Years','Total Population: 45 to 54 Years','Total Population: 55 to 64 Years','Total Population: 65 to 74 Years','Total Population: 75 to 84 Years','Total Population: 85 Years and Over']].astype('int64')

In [48]:
df_merged['%Total University'] = ((df_merged['Population 25 Years and Over: Some College or More'] - df_merged['Population 25 Years and Over: Doctorate Degree'])/df_merged['Population 25 Years and Over:'])
df_merged['%Total High school'] = ((df_merged["Population 25 Years and Over: High School Graduate or More (Includes Equivalency)"]- df_merged['Population 25 Years and Over: Some College or More'])/df_merged['Population 25 Years and Over:'])
df_merged['%Total Less than High School']= (df_merged['Population 25 Years and Over: Less than High School']/df_merged['Population 25 Years and Over:'])
df_merged['%Total Child Population'] = ((df_merged[ 'Total Population: Under 5 Years']+ df_merged['Total Population: 5 to 9 Years']+ df_merged['Total Population: 10 to 14 Years']+df_merged['Total Population: 15 to 17 Years'])/df_merged['Total Population:'])
df_merged['%Total Adult Population'] = ((df_merged[ 'Total Population: 18 to 24 Years']+ df_merged['Total Population: 25 to 34 Years']+ df_merged['Total Population: 35 to 44 Years']+df_merged['Total Population: 45 to 54 Years']+ df_merged['Total Population: 55 to 64 Years'])/df_merged['Total Population:'])
df_merged['%Total Elderly Population'] = ((df_merged['Total Population: 65 to 74 Years']+ df_merged['Total Population: 75 to 84 Years']+ df_merged['Total Population: 85 Years and Over'])/df_merged['Total Population:'])
df_merged['%Population with Disability']=(df_merged['E_DISABL']/df_merged['E_TOTPOP'])
df_merged['%LIMENG']=(df_merged['E_LIMENG']/df_merged['E_TOTPOP'])
df_merged['%NOINT']=(df_merged['E_NOINT']/df_merged['E_TOTPOP'])
df_merged['%DAYPOP']=(df_merged['E_DAYPOP']/df_merged['E_TOTPOP'])

df_merged['Density_Medical_Facilities_Population']=df_merged['Count_Medical_Facilities']/df_merged['E_TOTPOP']
df_merged['Density_Medical_Facilities_Area']=df_merged['Count_Medical_Facilities']/df_merged['AREA_SQMI']
df_merged["Density_Transportation_Stops_Pop"]=df_merged['Count_Transportation_Stops']/df_merged['E_TOTPOP']
df_merged["Density_Transportation_Stops_Area"]=df_merged['Count_Transportation_Stops']/df_merged['AREA_SQMI']

df_merged["Stops_per_Facility"]=df_merged['Count_Transportation_Stops']/df_merged['Count_Medical_Facilities']

In [49]:
facilities["Point Coun"].sum()

179004.0

In [50]:
test = pd.DataFrame(df_merged.isna().sum())
test[test[0]>78]

,0
US,84757
Census Region,84757
Census Division,84757
State (Census Code),84757
County Subdivision (FIPS),84757
...,...
Density_Medical_Facilities_Population,1670
Density_Medical_Facilities_Area,1276
Density_Transportation_Stops_Pop,1554
Density_Transportation_Stops_Area,1276


In [51]:
df_merged["Stops_per_Facility"].replace([np.nan], 0, inplace=True)
#df_merged["Density_Medical_Facilities_Population"].replace([np.nan], -np.inf, inplace=True)
#df_merged["Density_Medical_Facilities_Area"].replace([np.nan], -np.inf, inplace=True)
#df_merged["Density_Transportation_Stops_Pop"].replace([np.nan], np.inf, inplace=True)
#df_merged["Density_Transportation_Stops_Area"].replace([np.nan], np.inf, inplace=True)

In [52]:
targets       = []#['MAX_Completeness_pct', 'Weeks2MAX', 'IDX_W2M']
spatial       = ['Population Density (Per Sq. Mile)', 'Area (Land)', 'Average Commute to Work (In Min)', 'AREA_SQMI', 'Density_Medical_Facilities_Population', 'Density_Medical_Facilities_Area', 'Density_Transportation_Stops_Pop', 'Density_Transportation_Stops_Area', 'Stops_per_Facility']
socioeconomic = ["Total Population","Total Population:","% Total Population: Male","% Total Population: Female","Total Population:.1","% Total Population: Under 5 Years","% Total Population: 5 to 9 Years","% Total Population: 10 to 14 Years","% Total Population: 15 to 17 Years","% Total Population: 18 to 24 Years","% Total Population: 25 to 34 Years","% Total Population: 35 to 44 Years","% Total Population: 45 to 54 Years","% Total Population: 55 to 64 Years","% Total Population: 65 to 74 Years","% Total Population: 75 to 84 Years","% Total Population: 85 Years and Over","% Total Population: White Alone","% Total Population: Black or African American Alone","% Total Population: American Indian and Alaska Native Alone","% Total Population: Asian Alone","% Total Population: Native Hawaiian and Other Pacific Islander Alone","% Total Population: Some Other Race Alone","% Total Population: Two or More Races","% Population 25 Years and Over: Less than High School","% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","% Population 25 Years and Over: Some College or More","% Population 25 Years and Over: Bachelor's Degree or More","% Population 25 Years and Over: Master's Degree or More","% Population 25 Years and Over: Professional School Degree or More","% Population 25 Years and Over: Doctorate Degree","% Population 16 Years and Over: in Labor Force","% Population 16 Years and Over: in Labor Force: in Armed Forces","% Population 16 Years and Over: in Labor Force: Civilian","% Population 16 Years and Over: in Labor Force: Civilian: Employed","% Population 16 Years and Over: in Labor Force: Civilian: Unemployed","% Population 16 Years and Over: Not in Labor Force","% Households: Less than $25,000","% Households: $25,000 to $49,999","% Households: $50,000 to $74,999","% Households: $75,000 to $99,999","% Households: $100,000 or More","Median Household Income (In 2021 Inflation Adjusted Dollars)","Gini Index","% Workers 16 Years and Over: Car, Truck, or Van","% Workers 16 Years and Over: Drove Alone","% Workers 16 Years and Over: Carpooled","% Workers 16 Years and Over: Public Transportation (Includes Taxicab)","% Workers 16 Years and Over: Motorcycle","% Workers 16 Years and Over: Bicycle","% Workers 16 Years and Over: Walked","% Workers 16 Years and Over: Other Means","% Workers 16 Years and Over: Worked At Home","% Total: No Health Insurance Coverage","% Total: with Health Insurance Coverage","% Total: with Health Insurance Coverage: Public Health Coverage","% Total: with Health Insurance Coverage: Private Health Insurance","E_TOTPOP","EP_POV150","EP_UNEMP","EP_HBURD","EP_NOHSDP","EP_UNINSUR","EP_AGE65","EP_AGE17","EP_DISABL","EP_SNGPNT","EP_LIMENG","EP_MINRTY","EP_MUNIT","EP_MOBILE","EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV150","EPL_UNEMP","EPL_HBURD","EPL_NOHSDP","EPL_UNINSUR","SPL_THEME1","RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","EPL_LIMENG","SPL_THEME2","RPL_THEME2","EPL_MINRTY","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV150","F_UNEMP","F_HBURD","F_NOHSDP","F_UNINSUR","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_LIMENG","F_THEME2","F_MINRTY","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_DAYPOP","EP_NOINT","EP_AFAM","EP_HISP","EP_ASIAN","EP_AIAN","EP_NHPI","EP_TWOMORE","EP_OTHERRACE","%Total Child Population",'%Total University','%Total High school','%Total Less than High School','%Total Adult Population','%Total Elderly Population',"%Population with Disability","%LIMENG","%NOINT",'%DAYPOP']
others        = ["Qualifying Name","File identification","State Postal Abbreviation","Summary Level","Geographic Component","Logical Record Number","US","Census Region","Census Division","State (Census Code)","State (FIPS Code)","County of current residence","County Subdivision (FIPS)","Place (FIPS Code)","Census Tract","Block Group","Consolidated City","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(Census)","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(FIPS)","American Indian Trust Land/Hawaiian Home Land Indicator","American Indian TribalSubdivision (Census)","American Indian TribalSubdivision (FIPS)","Alaska Native RegionalCorporation (FIPS)","Metropolitan and MicropolitanStatistical Area","Combined Statistical Area","Metropolitan Statistical Area-Metropolitan Division","Metropolitan Area Central City","Metropolitan/MicropolitanIndicator Flag","New England City and Town Area","New England City and TownCombined Statistical Area","New England City and Town AreaDivision","Urban Area","Urban Area Central Place","Current Congressional District ***","State Legislative District Upper","State Legislative District Lower","Voting District","ZIP Code Tabulation Area (3-digit)","5-digit ZIP Code Tabulation Area","Subminor Civil Division (FIPS)","State-School District (Elementary)","State-School District (Secondary)","State-School District (Unified)","Urban/Rural","Principal City Indicator","Traffic Analysis Zone","Urban Growth Area","Public Use Microdata Area â€“ 5%File","Public Use Microdata Area - 1% File","Geographic Identifier","Area Name","Tribal Tract","Tribal Block Group","Place (State FIPS + Place FIPS)","Total Population: Male","Total Population: Female","Total Population: Under 5 Years","Total Population: 5 to 9 Years","Total Population: 10 to 14 Years","Total Population: 15 to 17 Years","Total Population: 18 to 24 Years","Total Population: 25 to 34 Years","Total Population: 35 to 44 Years","Total Population: 45 to 54 Years","Total Population: 55 to 64 Years","Total Population: 65 to 74 Years","Total Population: 75 to 84 Years","Total Population: 85 Years and Over","Total Population:.2","Total Population: White Alone","Total Population: Black or African American Alone","Total Population: American Indian and Alaska Native Alone","Total Population: Asian Alone","Total Population: Native Hawaiian and Other Pacific Islander Alone","Total Population: Some Other Race Alone","Total Population: Two or More Races","Population 25 Years and Over:","Population 25 Years and Over: Less than High School","Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","Population 25 Years and Over: Some College or More","Population 25 Years and Over: Bachelor's Degree or More","Population 25 Years and Over: Master's Degree or More","Population 25 Years and Over: Professional School Degree or More","Population 25 Years and Over: Doctorate Degree","Population 16 Years and Over:","Population 16 Years and Over: in Labor Force","Population 16 Years and Over: in Labor Force: in Armed Forces","Population 16 Years and Over: in Labor Force: Civilian","Population 16 Years and Over: in Labor Force: Civilian: Employed","Population 16 Years and Over: in Labor Force: Civilian: Unemployed","Population 16 Years and Over: Not in Labor Force","Households:","Households: Less than $25,000","Households: $25,000 to $49,999","Households: $50,000 to $74,999","Households: $75,000 to $99,999","Households: $100,000 or More","Workers 16 Years and Over:","Workers 16 Years and Over: Car, Truck, or Van","Workers 16 Years and Over: Drove Alone","Workers 16 Years and Over: Carpooled","Workers 16 Years and Over: Public Transportation (Includes Taxicab)","Workers 16 Years and Over: Motorcycle","Workers 16 Years and Over: Bicycle","Workers 16 Years and Over: Walked","Workers 16 Years and Over: Other Means","Workers 16 Years and Over: Worked At Home","Total:","Total: No Health Insurance Coverage","Total: with Health Insurance Coverage","Total: with Health Insurance Coverage: Public Health Coverage","Total: with Health Insurance Coverage: Private Health Insurance","W","ST","STATE","ST_ABBR","STCNTY","COUNTY","LOCATION","M_TOTPOP","E_HU","M_HU","E_HH","M_HH","E_POV150","M_POV150","E_UNEMP","M_UNEMP","E_HBURD","M_HBURD","E_NOHSDP","M_NOHSDP","E_UNINSUR","M_UNINSUR","E_AGE65","M_AGE65","E_AGE17","M_AGE17","E_DISABL","M_DISABL","E_SNGPNT","M_SNGPNT","E_LIMENG","M_LIMENG","E_MINRTY","M_MINRTY","E_MUNIT","M_MUNIT","E_MOBILE","M_MOBILE","E_CROWD","M_CROWD","E_NOVEH","M_NOVEH","E_GROUPQ","M_GROUPQ","MP_POV150","MP_UNEMP","MP_HBURD","MP_NOHSDP","MP_UNINSUR","MP_AGE65","MP_AGE17","MP_DISABL","MP_SNGPNT","MP_LIMENG","MP_MINRTY","MP_MUNIT","MP_MOBILE","MP_CROWD","MP_NOVEH","MP_GROUPQ","E_NOINT","M_NOINT","E_AFAM","M_AFAM","E_HISP","M_HISP","E_ASIAN","M_ASIAN","E_AIAN","M_AIAN","E_NHPI","M_NHPI","E_TWOMORE","M_TWOMORE","E_OTHERRACE","M_OTHERRACE","MP_NOINT","MP_AFAM","MP_HISP","MP_ASIAN","MP_AIAN","MP_NHPI","MP_TWOMORE","MP_OTHERRACE","Count_Medical_Facilities","Count_Transportation_Stops"]

In [58]:
df_merged["County of current residence"].nunique()

302

## Reduce number of columns

In [ ]:
columns_to_keep=  targets + spatial + socioeconomic

In [ ]:
df_merged_full = df_merged.copy()
df_merged=df_merged[columns_to_keep].copy()

In [ ]:
#df_merged.dropna(inplace=True)

In [ ]:
pd.DataFrame(df_merged.isna().sum())

### Download merged DF

In [ ]:
#df_merged.to_excel('df_merged.xlsx')
df_merged.to_csv('../df_tracts.csv')